In [153]:
#!jupyter nbextension enable --py gmaps

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import time
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [154]:
# File to Load (Remember to Change These)
file_to_load = "output_data/cities.csv"

# Read Purchasing File and store into Pandas data frame
df = pd.read_csv(file_to_load, encoding="utf-8")

# Store csv created in part one into a DataFrame

df.insert(0, 'City_ID', range(0, len(df)))
df['City'] = df['City'].str.lower()
df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,arraial do cabo,-22.9661,-42.0278,84.20,74,0,8.05,BR,1616705605
1,1,jamestown,42.0970,-79.2353,68.00,37,1,8.08,US,1616705891
2,2,rikitea,-23.1203,-134.9692,79.54,73,61,18.41,PF,1616705845
3,3,brooks,50.5834,-111.8851,32.00,69,100,12.66,CA,1616705893
4,4,lavrentiya,65.5833,-171.0000,3.00,84,28,2.46,RU,1616705894


In [156]:
print(f"The number of hotels in the sample is {len(df)}")

The number of hotels in the sample is 578


In [157]:
clean_df = df.dropna()
print(f"The number of hotels in the updated (no NA values) sample is {len(clean_df)}") 

The number of hotels in the updated (no NA values) sample is 574


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [158]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = clean_df[["Lat", "Lng"]].astype(float)

# Convert humidity to float and store
humiditymap = clean_df["Humidity"].astype(float)

# Heatmap of humidity
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humiditymap, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
    * A max temperature lower than 80 degrees but higher than 70.

    * Wind speed less than 10 mph.

    * Zero cloudiness.

* Drop any rows will null values.

In [159]:
vacay_df = clean_df.loc[(clean_df["Max Temp"] > 70) & (clean_df["Max Temp"] < 80)]
vacay_df = clean_df.loc[(clean_df["Wind Speed"] < 10)]
vacay_df = clean_df.loc[(clean_df["Cloudiness"] == 0)]
vacay_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,arraial do cabo,-22.9661,-42.0278,84.20,74,0,8.05,BR,1616705605
8,8,port macquarie,-31.4333,152.9167,63.00,82,0,1.01,AU,1616705900
36,36,taoudenni,22.6783,-3.9836,72.81,10,0,11.45,ML,1616705935
40,40,paris,48.8534,2.3488,51.01,66,0,3.44,FR,1616705353
50,50,saint-philippe,-21.3585,55.7679,77.00,83,0,3.44,RE,1616705951
...,...,...,...,...,...,...,...,...,...,...
529,529,iisalmi,63.5592,27.1907,35.60,80,0,6.91,FI,1616706587
535,535,iquique,-20.2208,-70.1431,71.60,64,0,20.71,CL,1616706595
545,545,burgos,42.3394,-3.7079,48.20,57,0,3.44,ES,1616706582
558,558,mühlacker,48.9500,8.8333,51.01,62,0,3.44,DE,1616706625


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [161]:
# Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

# Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

hotel_df = vacay_df.drop(columns=['City_ID', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed', 'Date'])
# vacay_hotel_df.columns.tolist()
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng']]
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,arraial do cabo,BR,-22.9661,-42.0278,
8,port macquarie,AU,-31.4333,152.9167,
36,taoudenni,ML,22.6783,-3.9836,
40,paris,FR,48.8534,2.3488,
50,saint-philippe,RE,-21.3585,55.7679,
...,...,...,...,...,...
529,iisalmi,FI,63.5592,27.1907,
535,iquique,CL,-20.2208,-70.1431,
545,burgos,ES,42.3394,-3.7079,
558,mühlacker,DE,48.9500,8.8333,


In [ ]:
# find the closest hotel of each type to coordinates

# Set parameters to search for a hotel

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "target_radius": 5000,
    "rankby": "distance",
    "keyword": "hotel",
    "key": g_key,
}

# Iterate through 
for index, row in hotel_df.iterrows():
    
    # use iterrows to iterate through pandas dataframe
    # get city name from df
    city = row['City']
    lat = row["Lat"]
    lng = row["Lng"]
    #     lat = row["Lat"].astype(float)
    #     lng = row["Lng"].astype(float)

    # add keyword to params dict
    params['location'] = (f"{lat}, {lng}")
    
    # Use the search term: "Hotel" and our lat/lng
    # make request and print url
    # convert to json
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']  
    
    # Grab the first hotel from the results and store the name 
    try:
        print(f"Closest hotel is {results[0]['name']}.")
  
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, "Lat"] = results[0]["geometry"]["location"]["lat"]
        hotel_df.loc[index, "Lng"] = results[0]["geometry"]["location"]["lng"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    time.sleep(1.01)
    print("------------")

hotel_df.head()

# NOTE: Do not change any of the code in this cell

In [165]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
                            info_box_content=[f"{rate}" for rate in hotel_info])
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))